# Installation

In [1]:
!pip install wandb -qqq > /dev/null
import wandb

wandb.login()

wandb: Currently logged in as: 367121 (house-666). Use `wandb login --relogin` to force relogin


True

In [2]:
!rm -rf modpo

!git clone https://github.com/ZHZisZZ/modpo.git > /dev/null

Cloning into 'modpo'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 164 (delta 67), reused 154 (delta 57), pack-reused 0 (from 0)
Receiving objects: 100% (164/164), 58.64 KiB | 4.51 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [3]:
# conda create -n modpo python=3.10
# conda activate modpo
# !pip install torch==2.1.0 --index-url https://download.pytorch.org/whl/cu118
# !pip install -r requirements.txt
# # (optional) pip install flash-attn==2.3.2 --no-build-isolation

In [4]:
# Colab version for CPU, else modify code above

# > /dev/null - to hide messages, but watch warning and errors
# !pip install virtualenv > /dev/null
# !virtualenv modpo > /dev/null

# !source modpo/bin/activate > /dev/null

# !pip install torch==2.1.0 --index-url https://download.pytorch.org/whl/cu118 > /dev/null

# !pip install -r modpo/requirements.txt > /dev/null

# !pip install flash-attn==2.3.2 --no-build-isolation > /dev/null

In [5]:
# another collab version for T4

!pip uninstall -y torch torchaudio torchvision fsspec gcsfs datasets bigframes trl > /dev/null

!pip install torch==2.4.1 torchaudio==2.4.1 torchvision==0.19.1 --index-url https://download.pytorch.org/whl/cu121 > /dev/null
!pip install fsspec==2023.6.0 gcsfs>=2023.3.0 > /dev/null

!pip install datasets==2.14.5 > /dev/null

!pip install -r modpo/requirements.txt > /dev/null

# Обучение авторской модели

In [6]:
# import os

# os.chdir("modpo")

# # modified num_processes: 8 -> 1
# # edit your number!
# !PYTHONPATH=. accelerate launch --config_file scripts/accelerate_configs/multi_gpu.yaml --num_processes=1 \
#     scripts/examples/dpo/dpo.py \
#     --sft_model_name "PKU-Alignment/alpaca-7b-reproduced" \
#     --prompt_template "BEGINNING OF CONVERSATION: USER: {raw_prompt} ASSISTANT:" \
#     --dataset_name "PKU-Alignment/PKU-SafeRLHF-10K-safer" \
#     --max_length 512 \
#     --training_args.output_dir "./output/PKU-Alignment/PKU-SafeRLHF-10K/modpo/rm/safer" \
#     --training_args.run_name "PKU-Alignment/PKU-SafeRLHF-10K/modpo/rm/safer" \
#     --training_args.per_device_train_batch_size 6 \
#     --training_args.per_device_eval_batch_size 6 \
#     --training_args.gradient_accumulation_steps 2 \
#     --training_args.learning_rate 5e-4 \
#     --peft_config.r 64 \
#     --peft_config.target_modules q_proj k_proj v_proj o_proj \
#     --peft_config.lora_alpha 1 \
#     --peft_config.lora_dropout 0

# Тест на маленьком датасете и низком качестве гиперпараметров

In [7]:
# import shutil
# import os

# cache_dir = os.path.expanduser("~/.cache/huggingface/datasets")
# shutil.rmtree(cache_dir)
# print(f"Кэш Hugging Face очищен: {cache_dir}")

In [8]:
# # Загрузка основного датасета
# from datasets import load_dataset

# dataset = load_dataset("PKU-Alignment/PKU-SafeRLHF-10K")

# print(dataset)

In [9]:
# num_examples = len(dataset["train"])
# subset_size = int(num_examples * 0.01)

# small_subset = dataset["train"].shuffle(seed=42).select(range(subset_size))

# print(small_subset)

In [10]:
import os

os.chdir("modpo")

!PYTHONPATH=. accelerate launch --config_file scripts/accelerate_configs/multi_gpu.yaml --num_processes=1 \
    scripts/examples/dpo/dpo.py \
    --sft_model_name "PKU-Alignment/alpaca-7b-reproduced" \
    --prompt_template "BEGINNING OF CONVERSATION: USER: {raw_prompt} ASSISTANT:" \
    --dataset_name "PKU-Alignment/PKU-SafeRLHF-10K" \
    --max_length 256 \
    --training_args.output_dir "./output/PKU-Alignment/PKU-SafeRLHF-10K/modpo/rm/safer" \
    --training_args.run_name "PKU-Alignment/PKU-SafeRLHF-10K/modpo/rm/safer" \
    --training_args.per_device_train_batch_size 1 \
    --training_args.per_device_eval_batch_size 1 \
    --training_args.gradient_accumulation_steps 1 \
    --training_args.learning_rate 5e-4 \
    --peft_config.r 8 \
    --peft_config.lora_alpha 1 \
    --peft_config.lora_dropout 0

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
2024-10-14 14:24:42.972347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 14:24:42.992464: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 14:24:42.998669: E external/local_xla/xla/stream_execu